In [6]:
import numpy as np
import matplotlib.pyplot as plt
import preprocessing as pp
import implementations as imp

# Import Data Sets


In [7]:
path="/Users/leocusumano/myCloud/EPFL/MA1/ML/Proj1/DATASETS/f_sets/"

x_train=np.load(path+"f_x_train_.npy")
x_test=np.load(path+"f_x_test_.npy")
y_train=np.load(path+"f_y_train_.npy")
test_ids=np.load(path+"f_test_ids_.npy")
train_ids=np.load(path+"f_train_ids_.npy")

In [8]:

x,x_t,y=pp.Edited_clean_data(x_train,y_train,x_test,correlation_thr=.95,nan_thr=.8,std_thr=0.1)

X,Y,X_T,Y_T=pp.split_data(x,y,.8)

Number of rows dropped because of a NaN score > 0.8:  71344
Number of columns dropped because of a NaN score > 0.8:  121
Number of columns with std < 0.1: 8
Number of columns with correl_coef > 0.95: 19
Handling NaN values...
Number of columns with corr_coef> 0.95 after cleaning: 2
The data has been cleaned and standardized
The cleaned x-data has the following shape:  (256791, 171)
The cleaned y-data has the following shape:  (256791,)
The cleaned x-data-to-predict has the following shape:  (109379, 171)


# Logistic

In [9]:
param_grid = {
    "max_iters": [500, 1000, 1500],
    "gamma": [0.001, 0.01, 0.1],
    "lambda_": [0.01, 0.1, 1],
}

In [10]:
best_params_log = None
best_score_log = float("inf")
w_initial = np.zeros(X.shape[1])
best_w_log = w_initial

for max_iters in param_grid["max_iters"]:
    for gamma in param_grid["gamma"]:
            w, loss = imp.logistic_regression(Y, X, w_initial, max_iters, gamma)
            print("Loss:", loss)
            if loss < best_score_log:
                best_score_log = loss
                best_w_log = w
                best_params_log = {
                    "max_iters": max_iters,
                    "gamma": gamma,
                }
                
                
print("Best hyperparameters found: ", best_params_log)

Loss: 0.43139030377581666


/Users/leocusumano/myCloud/EPFL/MA1/ML/Proj1/ML-proj1/implementations.py:342: RuntimeWarning: divide by zero encountered in log
  * np.sum(y * np.log(sigmoid(tx @ w)) + (1 - y) * np.log(1 - sigmoid(tx @ w)))
/Users/leocusumano/myCloud/EPFL/MA1/ML/Proj1/ML-proj1/implementations.py:342: RuntimeWarning: invalid value encountered in multiply
  * np.sum(y * np.log(sigmoid(tx @ w)) + (1 - y) * np.log(1 - sigmoid(tx @ w)))


Loss: nan
Loss: nan
Loss: 0.2661288555019352
Loss: nan
Loss: nan
Loss: 0.13395547104010802
Loss: nan
Loss: nan
Best hyperparameters found:  {'max_iters': 1500, 'gamma': 0.001}


In [17]:
w_tot, loss_tot = imp.logistic_regression(y, x, w_initial, 1500, 0.001)

In [22]:
from sklearn.metrics import f1_score,roc_auc_score,roc_curve

# Compute the predicted probabilities
y_pred_proba = imp.sigmoid(x @ w_tot)

# Search for the best threshold
thresholds = np.arange(0.0, 1.0, 0.01)
f1_scores = []

for threshold in thresholds:
    y_pred_log = np.where(y_pred_proba > threshold, 1, -1)
    f1 = f1_score(y, y_pred_log)
    f1_scores.append(f1)

# Find the threshold with the highest F1 score
optimal_threshold = thresholds[np.argmax(f1_scores)]
print("Optimal Threshold based on F1 score:", optimal_threshold, "F1 score:", np.max(f1_scores))





Optimal Threshold based on F1 score: 0.49 F1 score: 0.3021654567012668


In [24]:
import helpers as hlp

In [27]:
y_sub = imp.sigmoid(x_t @ w_tot)
y_sub = np.where(y_sub > .49, 1, -1)

hlp.create_csv_submission(test_ids, y_sub, "/Users/leocusumano/myCloud/EPFL/MA1/ML/Proj1/DATASETS/y_pred_logg.csv")

